<a href="https://colab.research.google.com/github/canerskrc/Borusan_Kod_Adi_Dijital/blob/main/borusan_day13_decisiontree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1- Bilgi Kazanımı

X ={a,a,a,a,b,b,b,b,b,b,b }

a = 4 adet
b= 7 adet

Entropi = 4/11 .log(4/11) + 7/11.log(7/11)

#Gini İndeks

bir düğümde seçilen iki özellik farklı sınıfa düşer mi? sorusunun cevabıdır.

Gini küçükse = aynı sınıfa düşme ihtimali yüksek ( düğüm saf )
gini büyükse = farklı sınıfa düşme ihtimali yüksek ( karışık düğüm )

#Ağaç Algoritma Türleri

1- CART ( Classification and Regression Trees )

2- MARS ( Multivariate Adaptive Regression Splines )

3- Conditional Inference Trees

4- CHAID ( Chi-Square Automatic Interaction Detection )

5- ID3 ( Iteratif Dichotomiser 3 )



In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv("train.csv")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              76518 non-null  int64  
 1   Marital status                                  76518 non-null  int64  
 2   Application mode                                76518 non-null  int64  
 3   Application order                               76518 non-null  int64  
 4   Course                                          76518 non-null  int64  
 5   Daytime/evening attendance                      76518 non-null  int64  
 6   Previous qualification                          76518 non-null  int64  
 7   Previous qualification (grade)                  76518 non-null  float64
 8   Nacionality                                     76518 non-null  int64  
 9   Mother's qualification                 

In [14]:
import math

X = df.drop("Target", axis=1)
y = df["Target"]


In [15]:
def entropy(y):
  values, counts = np.unique(y, return_counts= True)
  probs = counts/ counts.sum()
  return -np.sum(probs * np.log2(probs))

def gini(y):
  values, counts = np.unique(y, return_counts=True)
  probs = counts / counts.sum()
  return 1 - np.sum(probs ** 2 )

In [16]:
def best_split(X,y,criterion="gini"):
  best_feature, best_threshold, best_impurity = None, None, float("inf")
  impurity_func = gini if criterion == "gini" else entropy
  for feature in X.columns:
    unique_values = X[feature].unique()

    for value in unique_values:

      if np.issubdtype(X[feature].dtype,np.number):
        left_mask = (X[feature] <= value)
      else:
        left_mask = (X[feature] == value)
      right_mask = ~left_mask #sol tarafta olmayan veriler sağ tarafa gider.

      if left_mask.sum() == 0 or right_mask.sum() == 0:
        continue

      left_impurity = impurity_func(y[left_mask])
      right_impurity = impurity_func(y[right_mask])

      weighted_impurity = ((left_mask.sum()/len(y)) * left_impurity +
                           (right_mask.sum()/len(y)) * right_impurity)

      if weighted_impurity < best_impurity:
        best_feature = feature
        best_threshold = value
        best_impurity = weighted_impurity

  return best_feature, best_threshold, best_impurity

#Seçilen 5 özelliğe göre en iyi split'i bulma

In [17]:
selected_features = ["Curricular units 2nd sem (grade)",
                     "Curricular units 1st sem (grade)",
                     "Previous qualification (grade)",
                     "Admission grade",
                     "Age at enrollment"]

X_selected = X[selected_features]

best_feature, best_threshold, best_impurity = best_split(X_selected, y, criterion="gini")
print("En iyi özellik: ", best_feature)
print("Eşik değer:" , best_threshold)
print("Impurity: ", best_impurity)


En iyi özellik:  Curricular units 2nd sem (grade)
Eşik değer: 10.5
Impurity:  0.43577337913793224


#Karar Ağacı Node Sınıfı

In [18]:
class Node:
  def __init__(self, feature= None, threshold= None,  left = None, right=None, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value #yaprak tahmini

#Ağaç Kurulumu ( Recursive )

In [19]:
def build_tree(X,y, depth=0, max_depth=3, criterion="gini"):
  if len(np.unique(y)) == 1 or depth >= max_depth:
    values, counts = np.unique(y, return_counts=True)
    return Node(value=values[np.argmax(counts)])
  feature, threshold = best_split(X,y, criterion=criterion)
  if feature is None:
    values,counts = np.unique(y, return_counts=True)
    return Node(value=values[np.argmax(counts)])
  if X[feature].dtype == "object":
    left_mask = (X[feature] == threshold)
  else:
    left_mask = (X[feature] <= threshold)
  right_mask = ~left_mask
#recursive çağrı
  left = build_tree(X[left_mask],y[left_mask], depth+1, max_depth, criterion)
  right = build_tree(X[right_mask],y[right_mask], depth+1, max_depth, criterion)

  return Node(feature, threshold, left,right, )

In [20]:
import matplotlib.pyplot as plt
import networkx as nx

def plot_tree(node, graph=None, parent=None, edge_label=""):
  if graph is None:
    graph = nx.DiGraph()
  if node.value is not None:
    label =f"Leaf:{node.value}"
  else:
    label = f"{node.feature } <= {node.threshold}"
  graph.add_node(id(node), label=label)

  if parent:
    graph.add_edge(parent, id(node),label=edge_label)
  if node.left:
    plot_tree(node.left, graph, id(node), "Yes")
  if node.right:
    plot_tree(node.right, graph, id(node),"No")
  return graph


def draw_graph(graph):
  pos = nx.spring_layout(graph)
  labels = nx.get_node_attributes(graph, "label")
  edge_labels = nx.get_edge_attributes(graph,"label")

  nx.draw(graph, pos, with_labels=False, node_size=2000, node_color="lightblue")
  nx.draw_networkx_labels(graph, pos, labels)
  nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
  plt.show()

In [22]:
tree = build_tree(X_selected, y, max_depth=3, criterion="gini")
graph = plot_tree(tree)
draw_graph(graph)


ValueError: too many values to unpack (expected 2)

Split Mantığı

- Gini = Daha hızlı, genellikle CART algoritmalarda production ortamında kullanılır.

- Entropy = Daha hassas, ama logaritma içerdiğinden maliyetli.

- max_depth = veri büyüklüğü, özellik sayısı, sınıf dağılımları, hedef performansı, Grid search / Cross Validation teknikleriyle belirlenmesi daha gerçekçi olur.

- min_samples_split = bir düğümün bölünebilmesi için gereken örnek sayısı.

- min_samples_leaf = Yaprakta kalması gereken minimum örnek sayısı.

- Kategorik verilerde yüksek kardinaliteli özellikler bir kolona ağaç takılır ve anlamsız bölmeler gerçekleşir. Bu tip kolonlarda önceden gruplamak( binning ) Ya da one-hot encoding yerine target encoding kullanılır.

- Overfitting ihtimalinde çok derin ağaçlarda pruning ( budama ) ile fazla dalları sonradan kesmek gerekir. Random forest veya Gradient Boosting algoritmalarında tek ağaç yerine ensemble yöntemleri kullanmak daha mantıklı olur. ( Ağaç tabanlı topluluk yönetimi XGBoost, CatBoost, LightGBM )



